In [72]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import norm

import matplotlib.pyplot as plt

In [73]:
downweight_granularity=['dtime','Direction','dday','month']

# Data Import

In [74]:
Data=pd.read_csv('R_Output_Test.csv',sep=',')

In [75]:
Data=Data[['DepDate','yday','dtime','Direction','month','dday','Dprio','NumPax']]

In [76]:
Map_Dates=DataFrame([Data['DepDate'].unique().tolist(),[pd.to_datetime(x) for x in Data['DepDate'].unique().tolist()]]).transpose()
Map_Dates.columns=['DepDate','DepDate_dt']
Map_Dates['DepDate']=Map_Dates['DepDate'].astype('str')

Data=Data.merge(Map_Dates,on=['DepDate'])
Data['DepDate']=Data['DepDate_dt']
Data=Data[[x for x in Data.columns if x!='DepDate_dt']]

In [77]:
Data['TicketDate']=Data['DepDate']-Data['Dprio'].apply(lambda x: datetime.timedelta(x))

In [78]:
Data=Data.loc[Data['TicketDate']<=pd.to_datetime('2019-03-27'),:]

In [79]:
Data=Data[[x for x in Data.columns if x!='TicketDate']]

In [80]:
Data['Dprio']=-Data['Dprio']
Data=Data.set_index(['DepDate','yday','dtime','Direction','month','dday','Dprio'])
Data=Data.groupby(level=[0,1,2,3,4,5,6]).sum().groupby(level=[0,1,2,3,4,5])['NumPax'].cumsum().reset_index()

In [81]:
Data['Dprio']=-Data['Dprio']

In [82]:
IdealCurve=pd.read_csv('IdealCurve.csv')

In [83]:
Data=Data.merge(IdealCurve,on=downweight_granularity+['Dprio'])

In [84]:
Data=Data[['DepDate','dtime','Direction','dday','Dprio','NumPax','Ideal_lower','Ideal_average_downweighted','Ideal_average_full','Ideal_upper']]\
.sort_values(by=['DepDate','Direction','dtime','Dprio'])

In [85]:
Data['NumPax']=Data['NumPax'].apply(lambda x: int(round(x)))
Data['Ideal_lower']=Data['Ideal_lower'].apply(lambda x: int(round(x)))
Data['Ideal_average_downweighted']=Data['Ideal_average_downweighted'].apply(lambda x: int(round(x)))
Data['Ideal_average_full']=Data['Ideal_average_full'].apply(lambda x: int(round(x)))
Data['Ideal_upper']=Data['Ideal_upper'].apply(lambda x: int(round(x)))

In [86]:
Data.to_csv('GUI_in.csv',index=False)

In [87]:
Data.loc[Data['DepDate']==pd.to_datetime('2019-04-01'),:]

,DepDate,dtime,Direction,dday,Dprio,NumPax,Ideal_lower,Ideal_average_downweighted,Ideal_average_full,Ideal_upper
3469,2019-04-01,960,I,Monday,5,22,39,44,62,65
3468,2019-04-01,960,I,Monday,6,22,37,43,60,64
3467,2019-04-01,960,I,Monday,7,21,35,42,59,63
3466,2019-04-01,960,I,Monday,8,20,34,41,57,61
3465,2019-04-01,960,I,Monday,9,20,32,39,55,60
3464,2019-04-01,960,I,Monday,10,18,31,38,54,59
3463,2019-04-01,960,I,Monday,11,20,30,37,52,58
3461,2019-04-01,960,I,Monday,12,20,28,36,51,56
3459,2019-04-01,960,I,Monday,13,20,27,35,49,55
3457,2019-04-01,960,I,Monday,14,19,26,34,48,54
